# Implement AdditivePhylogeny

The following recursive algorithm, called AdditivePhylogeny, finds the simple tree fitting an n x n additive distance matrix D. We assume that you have already implemented a program Limb(D, j) that computes LimbLength(j) for a leaf j based on the distance matrix D. Rather than selecting an arbitrary leaf j from Tree(D) for trimming, AdditivePhylogeny selects leaf n (corresponding to the last row and column of D).

In [ ]:
import numpy as np

x=[]
d=[]

with open('rosalind_ba7c.txt','r') as f :
    x = []
    for line in f:
        line = line.strip()
        x.append(line)

    tool = int(x[0])
    for i in range(tool):
        d.append([])
        for j in range(tool):
            d[i].append(0)

    for i in range(tool):
        s = x[i + 1].split()
        for k in range(tool):
            d[i][k] = int(s[k])

def dfs(g, f, d, v):
    v[f] = 1
    for j, i in g[f]:
        if v[j]:
            continue

        if j == d:
            return [(f, i), (d, 0)]

        path = dfs(g, j, d, v)
        if path is not None:
            return [(f, i)] + path

    return None

def additive(D, n, a, nv):
    if n == 2:

        if 0 in a:
            a[0].append((1, D[0][1]))
        else:
            a[0] = [(1, D[0][1])]

        if 1 in a:
            a[1].append((0, D[0][1]))
        else:
            a[1] = [(0, D[0][1])]
        return a

    n = n - 1

    q = float('inf')
    for i in range(len(D)):
        for k in range(len(D[i])):
            if (i != n) and (n != k) and (i != k):
                result = (D[i][n] + D[n][k] - D[i][k]) / 2
                q = min(q,q, result)

    q=int(q)
    for j in range(n):
        D[j][n] -= q
        D[n][j] = D[j][n]

    sayer = [i for i in range(n)]
    for j in range(len(sayer) - 1):
        i = sayer[j]
        for m in range(j + 1, len(sayer)):
            k = sayer[m]
            if D[i][n] + D[n][k] == D[i][k]:
                choice_i = i
                choice_k = k
    x = D[choice_i][n]
    del D[-1]
    for i in range(len(D)):
        del D[i][-1]

    while nv in a:
        nv += 1
    T = additive(D, n, a, nv)

    V = -1
    vis=[False] * (max(T) + 1)
    s = dfs(T, choice_i, choice_k, vis)
    dist = 0
    for j, (node, weight) in enumerate(s[:-1]):
        dist = weight+dist
        if dist == x:
            V = s[j + 1]

    if V == -1:
        V = nv
        while V in T:
            V += 1
        dist = 0
        j = 0
        while dist < x:
            l = dist
            dist = s[j][1]+dist
            j = j+1

        if s[j - 1][0] in a:
            for i, (node, _) in enumerate(a[s[j - 1][0]]):
                if node == s[j][0]:
                    del a[s[j - 1][0]][i]

        if s[j][0] in a:
            for i, (node, _) in enumerate(T[s[j][0]]):
                if node == s[j - 1][0]:
                    del T[s[j][0]][i]


        if V in T:
            T[V].append((s[j][0],  dist - x))
        else:
            T[V] = [(s[j][0],  dist - x)]

        if s[j][0] in T:
            T[s[j][0]].append((V,  dist - x))
        else:
            T[s[j][0]] = [(V,  dist - x)]

        if V in T:
            T[V].append((s[j - 1][0], x - l))
        else:
            T[V] = [(s[j - 1][0], x - l)]

        if s[j - 1][0] in T:
            T[s[j - 1][0]].append((V, x - l))
        else:
            T[s[j - 1][0]] = [(V, x - l)]


    if V in T:
        T[V].append((n, q))
    else:
        T[V] = [(n, q)]

    if n in T:
        T[n].append((V, q))
    else:
        T[n] = [(V, q)]

    return T

a = {}
T = additive(d, tool, a, tool)
for i in T:
    for j, w in T[i]:
        print(f'{i}->{j}:{w}')